# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [29]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/home/siyuan/Documents/udemy-data-science/DataScience-Python3/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/home/siyuan/Documents/udemy-data-science/DataScience-Python3/emails/ham', 'ham'))

data = data.sample(frac=1)

training_data = data[:2000]
test_data = data[2000:]

Let's have a look at that DataFrame:

In [30]:
test_data.head()

,class,message
/home/siyuan/Documents/udemy-data-science/DataScience-Python3/emails/spam/00253.83b95b05e275286eddcf557ea581e754,spam,There is NO stumbling on to it! \n\n\n\nThe gr...
/home/siyuan/Documents/udemy-data-science/DataScience-Python3/emails/ham/01455.6f59785ae19e2d0c92f0f18764e5e8a6,ham,">>>>> ""CW"" == Carlo Wood <carlo@alinoe.com> wr..."
/home/siyuan/Documents/udemy-data-science/DataScience-Python3/emails/ham/02100.9e565da6180ec3bcba22c6de299c27a6,ham,"URL: http://www.newsisfree.com/click/-2,841779..."
/home/siyuan/Documents/udemy-data-science/DataScience-Python3/emails/ham/01912.a26e261a1b2b9af7d0c2a4341f173786,ham,URL: http://boingboing.net/#85490092\n\nDate: ...
/home/siyuan/Documents/udemy-data-science/DataScience-Python3/emails/ham/01970.953f97d05583f057ab9a56bf1e8a9087,ham,"URL: http://www.newsisfree.com/click/-4,804534..."


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [31]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(training_data['message'].values)

classifier = MultinomialNB()
targets = training_data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [46]:
test_counts = vectorizer.transform(test_data['message'].values)
predictions = classifier.predict(test_counts)
success = 0
for index, prediction in enumerate(predictions):
    if prediction == test_data['class'].values[index]:
        success += 1
correctRate = success / test_data.shape[0]
correctRate

0.948

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.